<h1>Land Feature CNN</h1>

Code and network framework gathered from:
https://www.tensorflow.org/tutorials/images/cnn

Authors:
- Thao Pham
- Lawrence Hoerst


In [ ]:
import converter
# This will load the colored images into np arrays and pickle them
converter.main()

In [4]:
# Import tensorflow for the model
import tensorflow as tf
# Import matplotlib to show retults
import matplotlib.pyplot as plt
# Import pickle to read in training data
import pickle

2.10.0


In [ ]:
# Class names to plot the images:
class_names = ['Water', 'Buildings', 'Roads', 'Foliage', 'Mineral deposits', 'Mountainous terrain', 'Rocky terrain', 'Sandy terrain', 'Plains', 'Snow', 'Grass']
class_colors = ['#0f5e9c', ('#f2f2f2', '#606060'), '#c4c4c4', '#3a5f0b', '#490e0e', '#5a7a4c', '#698287', '#f7ae64', '#c89e23', '#fffafa', '#7cfc00']
class_codes = {class_names[i]: i for i in range(len(class_names))}

# hex_to_rgb
cvt = lambda hex: ImageColor.getcolor(hex, "RGB")
colors_to_code = {cvt('#0f5e9c'): 0,
                  cvt('#f2f2f2'): 1, cvt('#606060'): 1,
                  cvt('#c4c4c4'): 2,
                  cvt('#3a5f0b'): 3,
                  cvt('#490e0e'): 4,
                  cvt('#5a7a4c'): 5,
                  cvt('#698287'): 6,
                  cvt('#f7ae64'): 7,
                  cvt('#c89e23'): 8,
                  cvt('#fffafa'): 9,
                  cvt('#7cfc00'): 10,
                  'default':      11}

In [2]:
# Instantiate the model
model = models.Sequential()
# Add layers
# We need to decide on how many layers we want
# These parameters are not yet setup for our network
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(200*200, activation='relu'))
model.add(layers.Dense(200*200))
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

NameError: name 'models' is not defined

In [1]:
print(model.summary())

NameError: name 'model' is not defined

In [7]:
class TwoWayDict(dict):
    """
    Class and functionality taken from stack overflow:
    https://stackoverflow.com/questions/1456373/two-way-reverse-map#:~:text=class%20TwoWayDict(,__len__(self)%20//%202
    """
    def __setitem__(self, key, value):
        # Remove any previous connections with these values
        if key in self:
            del self[key]
        if value in self:
            del self[value]
        dict.__setitem__(self, key, value)
        dict.__setitem__(self, value, key)

    def __delitem__(self, key):
        dict.__delitem__(self, self[key])
        dict.__delitem__(self, key)

    def __len__(self):
        """Returns the number of connections"""
        return dict.__len__(self) // 2

In [8]:
# unpickle our test data here
with open('training_inputs.pickle', 'rb') as inputsFile:
    training_inputs = pickle.load(inputsFile)
with open('training_outputs.pickle', 'rb') as outputsFile:
    training_outputs = pickle.load(outputsFile)

# unpickle the validation / testing outputs
with open('validation_inputs.pickle', 'rb') as inputsFile:
    validation_inputs = pickle.load(inputsFile)
with open('validation_outputs.pickle', 'rb') as outputsFile:
    validation_outputs = pickle.load(outputsFile)
    


Colored images converted successfully...
Raw images converted successfully...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 99, 99, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 48, 48, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 46, 46, 64)        36928     
                                            

ResourceExhaustedError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[135424,40000] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu [Op:Mul]

In [ ]:
# Train the model
history = model.fit(training_inputs, training_outputs, epochs=10, validation_data=(validation_inputs, validation_outputs))

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)